In [4]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_finetuning`


In [8]:
eng_file = "flores-22_dev/all/eng_Latn-mar_Deva/dev.eng_Latn"
mar_file = "flores-22_dev/all/eng_Latn-mar_Deva/dev.mar_Deva"
hin_file = "flores-22_dev/all/eng_Latn-hin_Deva/dev.hin_Deva"

In [9]:
import pandas as pd
output_file = "flores22_eng_mar_hin.csv"


# Read the files
with open(eng_file, "r", encoding="utf-8") as f:
    eng_lines = [line.strip() for line in f if line.strip()]

with open(mar_file, "r", encoding="utf-8") as f:
    mar_lines = [line.strip() for line in f if line.strip()]

with open(hin_file, "r", encoding="utf-8") as f:
    hin_lines = [line.strip() for line in f if line.strip()]
    
# Sanity check
if len(eng_lines) != len(mar_lines):
    print(f"Warning: Line count mismatch! ENG: {len(eng_lines)}, MAR: {len(mar_lines)}")
    min_len = min(len(eng_lines), len(mar_lines))
    eng_lines = eng_lines[:min_len]
    mar_lines = mar_lines[:min_len]

# Create DataFrame
df = pd.DataFrame({
    "src": eng_lines,
    "mar_Deva": mar_lines,
    "hin_Deva": hin_lines
})

# Save as CSV
df.to_csv(output_file, index=False, encoding="utf-8")

print(f"Saved {len(df)} sentence pairs to {output_file}")

Saved 997 sentence pairs to flores22_eng_mar_hin.csv


In [5]:
df.tail()

,src,mar_Deva,hin_Deva
992,The tourist season for the hill stations gener...,हिल स्टेशन्ससाठी पर्यटकांचा सीझन भारतात सामान्...,भारतीय हिल स्टशनों पर पर्यटकों का ज्यादा जमावड़...
993,"However, they have a different kind of beauty ...","परंतु, हिवाळ्यात त्यांच्याकडे एक भिन्न सौंदर्य...","हालांकि, सर्दियों के दौरान उनका एक अलग तरह का ..."
994,Only a few airlines still offer bereavement fa...,केवळ काही मोजक्या एयरलाईन्स बेरेवमेंट तिकीट दर...,केवल कुछ ही एयरलाइंस शोक किराए की पेशकश करती ह...
995,"Airlines that offer these include Air Canada, ...",ही सुविधा प्रदान करणार्‍या एअरलाईन्समध्ये यू.ए...,ये ऑफर देने वाली एयरलाइन में यूएस या कनाडा और ...
996,"In all cases, you must book by phone directly ...","या सर्व प्रकरणांमध्ये, तुम्ही फोनद्वारे थेट वि...","सभी मामलों में, आपको एयरलाइन की बुकिंग सीधे फ़..."


In [10]:
from datasets import Dataset, DatasetDict

# --- Step 3: Convert to Hugging Face Dataset ---
test_dataset = Dataset.from_pandas(df)

# Wrap it in a DatasetDict with a "test" split
dataset_dict = DatasetDict({"test": test_dataset})

# --- Step 4: Push to Hugging Face Hub as PRIVATE ---
repo_id = "thenlpresearcher/flores22_eng_mar_hin"

# Make sure you're logged in via `huggingface-cli login`
dataset_dict.push_to_hub(repo_id, private=False)

print(f"Uploaded dataset to https://huggingface.co/datasets/{repo_id}")

Creating parquet from Arrow format: 100%|████| 1/1 [00:00<00:00, 108.50ba/s]
Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.

Upload 0 LFS files: 0it [00:00, ?it/s]
Uploading the dataset shards: 100%|███████████| 1/1 [00:00<00:00,  1.31it/s]


Uploaded dataset to https://huggingface.co/datasets/thenlpresearcher/flores22_eng_mar_hin
